# pips

In [1]:
!pip install requests beautifulsoup4



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install LinkedinProfile

ERROR: Could not find a version that satisfies the requirement LinkedinProfile (from versions: none)
ERROR: No matching distribution found for LinkedinProfile

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
!pip install trafilatura


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 83.6 MB/s eta 0:00:00


In [2]:
!pip install requests beautifulsoup4

In [3]:
!pip install pandas

In [1]:
!pip install transformers trafilatura


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 65.9 MB/s eta 0:00:00


# codigo

In [1]:
import pandas as pd
import trafilatura
from transformers import pipeline
import time

# Caminho para o arquivo CSV
file_path = './database/finance_news.csv'

# Carrega o arquivo CSV em um DataFrame
df = pd.read_csv(file_path)

def extrair_texto_principal(url):
    try:
        # Baixar e extrair o texto principal da URL
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            texto_principal = trafilatura.extract(downloaded, output_format='txt')
            if texto_principal:
                return texto_principal
            else:
                return "Não foi possível encontrar o conteúdo principal."
        else:
            return "Erro ao baixar o conteúdo da página."
    except Exception as e:
        return f"Erro ao extrair texto de {url}: {e}"

# Inicializa o pipeline de summarization do Hugging Face
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def resumir_texto(texto, max_len=512, min_len=20):
    try:
        # Gera o resumo com comprimento ajustável
        resumo = summarizer(texto, max_length=max_len, min_length=min_len, do_sample=False)
        return resumo[0]['summary_text']
    except Exception as e:
        return f"Erro ao resumir o texto: {e}"

# Extrai a coluna 'links' e 'title' e converte para listas
urls = df['link'].tolist()
titles = df['title'].tolist()

# Lista para armazenar os dados da nova tabela
data = []

for title, url in zip(titles, urls):
    texto_completo = extrair_texto_principal(url)  # Extrai o texto principal da URL
    resumo_texto = resumir_texto(texto_completo, max_len=512, min_len=200)  # Ajusta o comprimento do resumo
    texto_final = f'{title}\n{resumo_texto}'  # Concatena o título com o resumo
    data.append({'news_text': texto_final})
    time.sleep(1)  # Espera 1 segundo entre as requisições

# Cria um novo DataFrame com os dados processados
df_final = pd.DataFrame(data, columns=['news_text'])

# Exibe a tabela final
print(df_final)

# Salvar a tabela em um novo arquivo CSV, se necessário
df_final.to_csv('./database/news_texts.csv', index=False)

/workspaces/SynaptaInvest/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspaces/SynaptaInvest/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 512, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 512, but your input_length is only 20. Since thi

                                            news_text
0   Abertura econômica é caminho para desenvolvime...
1   [Removed]\nPágina de conteúdo da pálina. Erro ...
2   [Removed]\nPágina de conteúdo da pálina. Erro ...
3   [Removed]\nPágina de conteúdo da pálina. Erro ...
4   Mais política industrial verde\nA demanda por ...
5   Alguém está ouvindo o que diz a Ciência?\nErro...
6   Alto Escalão: VR troca de comando\nSimone Marq...
7   Tecnologia como ferramenta de concretização do...
8   Ouribank oferece crédito para empresas para o ...
9   Ouribank oferece crédito para empresas para o ...
10  Post mente ao dizer que Starlink detém todos o...
11  Brasil desenha proposta para atrair investimen...
12  Brasil desenha proposta para atrair investimen...
13  Ecossistema do BB, BNDES e Itaúsa para investi...
14  Dexco obtém US$ 1 bilhão em financiamento para...
15  CDBs: prefixados e pós-fixados em movimentos o...
16  Starlink não é dona de todos os satélites do B...
17  “A diversidade acrescent

: 